In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read"

auth = SpotifyOAuth(client_id="75bb7a00dee045f694a684ca5617dd08",
                   client_secret="abdedf81e2bf4ee5accd0f7a1c8317c9",
                   redirect_uri="https://www.google.com/",
                   scope=scope)
sp = spotipy.Spotify(auth_manager=auth)

In [2]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [3]:
def extract_tracks_data(playlist_id):
    '''
    ids = []
    track_names = []
    artistGenres = []
    popularities = []
    print('-')
    playlist_tracks = sp.playlist_tracks(playlist_id=playlist_id)
    print('--')
    for item in playlist_tracks["items"]:
        if (item["track"] != None):
            artistId = item["track"]["album"]["artists"][0]["id"]
            ids.append(item["track"]["id"])
            popularities.append(item["track"]["popularity"])
            artistGenres.append(sp.artist(artistId)["genres"])
            track_names.append(item["track"]["name"])
    print("---")
    return ids, track_names, artistGenres, popularities
    '''
    playlist = pd.DataFrame(sp.playlist_tracks(playlist_id=playlist_id)["items"])
    playlist = pd.DataFrame(list(filter(lambda x : x != None, playlist["track"])))
    playlist["track_id"] = playlist["id"]
    artistIds = pd.DataFrame(
        pd.DataFrame(pd.DataFrame(playlist["album"].tolist())["artists"].tolist())[0].tolist())["id"]
    trackNames = playlist["name"]
    #artistGenres = list(map((lambda artistId : sp.artist(artistId)["genres"]), artistIds.tolist()))
    frame = pd.DataFrame()
    frame["genres"] = artistIds.apply((lambda artistId : sp.artist(artistId)["genres"]))
    frame["genres"] = frame["genres"].apply(lambda x : [y.replace(" ", "_") for y in x])
    
    return playlist.filter(["name","track_id", "popularity"]).join(artistIds).join(frame["genres"])

In [4]:
def extract_features(playlist_id):
    '''
    extracts features from a playlist
    '''
    '''
    ids, track_names, artistGenres, popularities = extract_tracks_data(playlist_id)
    print("hello")
    idToData = {} 
    data = sp.audio_features(tracks=ids)
    for i in range(0, len(track_names)):
        data[i]["genres"] = [x.replace(" ", "_") for x in artistGenres[i]]
        data[i]["popularity"] = popularities[i]
        idToData[track_names[i]] = data[i]
    '''
    df = extract_tracks_data(playlist_id)
    data = pd.DataFrame(sp.audio_features(tracks=df["track_id"].tolist()))
    df = df.join(data.filter(["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo"]))
    #df = pd.DataFrame(idToData).transpose()[["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo", "genres", "popularity"]]
    tf = TfidfVectorizer()
    df = df.join(pd.DataFrame(tf.fit_transform(df["genres"].apply((lambda x : " ".join(x)))).toarray(),
                     columns=tf.get_feature_names()))
    scaler = MinMaxScaler(feature_range=(0,1))
    for column in df.columns:
        badCols = tf.get_feature_names()
        badCols.append("genres")
        badCols.append("name")
        badCols.append("track_id")
        badCols.append("id")
        if (column not in badCols):
            data = df[column]
            df[column] = scaler.fit_transform(np.float32(data).reshape(len(data), 1))
    return df[["name", "track_id", "id","genres"]], df.loc[:, ~df.columns.isin(["name", "track_id", "id","genres"])]

In [5]:
def get_playlist_vector_and_features(playlist_url):
    info, data = extract_features(playlist_url)
    return info,data.sum(axis=0),data.columns 

In [6]:
from numpy import dot
from numpy.linalg import norm
def cosine_similarity(x,y):
    x = np.array(x)
    y = np.array(y)
    cos_sim = dot(x,y)/(norm(x)*norm(y))
    return cos_sim

In [7]:
def similar_songs(myPlaylistUrl, otherPlaylistUrl):
    myInfo,myVector, myFeatures = get_playlist_vector_and_features(myPlaylistUrl)
    otherPlaylistInfo,otherPlaylistFeatures = extract_features(otherPlaylistUrl)
    commonFeatures = otherPlaylistFeatures.columns.intersection(myFeatures)
    myVector = myVector[commonFeatures]
    otherPlaylistFeatures = otherPlaylistFeatures[commonFeatures]
    return otherPlaylistFeatures.apply(lambda x : cosine_similarity(myVector, 
                                                 x), axis = 1)

In [8]:
similar_songs("https://open.spotify.com/playlist/3OCVYiGIqURhQO5TVbrK7A?si=0b38a71016da464a",
             "https://open.spotify.com/playlist/0vvXsWCC9xrXsKd4FyS8kM?si=96fd28e8efc4481c")

AttributeError: 'NoneType' object has no attribute 'keys'